<a href="https://colab.research.google.com/github/Ricardojnf33/Modelos_Preditivos_Temperatura_Seatle-WA/blob/main/Perguntas_de_Negocio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Importação bibliotecas

In [ ]:
import pandas as pd
import re
import folium

from haversine import haversine
import plotly.express as px
import plotly.graph_objects as go

# Carregamento do dataframe

In [ ]:
!git clone https://github.com/Ricardojnf33/Pergunta_de_negocio_FTC.git

fatal: destination path 'Pergunta_de_negocio_FTC' already exists and is not an empty directory.


In [ ]:
df_raw = pd.read_csv('/content/Pergunta_de_negocio_FTC/Data/train.csv')

df_raw.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


# Limpeza

In [ ]:
# Fazendo uma cópia do DataFrame lido
df = df_raw.copy()

This will print the data type of the 'Time_taken(min)' column.

Finally, you can check the size of the groups in the groupby aggregation like this:

In [ ]:
# Check data types
print(df.dtypes)

ID                              object
Delivery_person_ID              object
Delivery_person_Age             object
Delivery_person_Ratings         object
Restaurant_latitude            float64
Restaurant_longitude           float64
Delivery_location_latitude     float64
Delivery_location_longitude    float64
Order_Date                      object
Time_Orderd                     object
Time_Order_picked               object
Weatherconditions               object
Road_traffic_density            object
Vehicle_condition                int64
Type_of_order                   object
Type_of_vehicle                 object
multiple_deliveries             object
Festival                        object
City                            object
Time_taken(min)                 object
dtype: object


In [ ]:
# Check data types
print(df['Time_taken(min)'].dtypes)

object


In [ ]:
# Check group sizes
print(df.groupby('City')['Time_taken(min)'].size())

City
Metropolitian     34093
NaN                1200
Semi-Urban          164
Urban             10136
Name: Time_taken(min), dtype: int64


In [ ]:
# Remover espaco da string // Com .strip() -> remove espacos em branco das extremidades da string
for i in range( len( df ) ):
    df.loc[i,'ID'] = df.loc[i,'ID'].strip()
    df.loc[i, 'Delivery_person_ID'] = df.loc[i, 'Delivery_person_ID'].strip()
    

# Excluir as linhas com a idade dos entregadores vazia
# ( Conceitos de seleção condicional)
linhas_vazias = df['Delivery_person_Age'] != 'NaN '
df = df.loc[linhas_vazias, :]

linhas_vazias = df['Road_traffic_density'] != 'NaN '
df = df.loc[linhas_vazias, :]

linhas_vazias = df['City'] != 'NaN '
df = df.loc[linhas_vazias, :]

linhas_vazias = df['Time_taken(min)'] != 'NaN '
df = df.loc[linhas_vazias, :]

linhas_vazias = df['Type_of_order'] != 'NaN '
df = df.loc[linhas_vazias, :]

linhas_vazias = df['Festival'] != 'NaN '
df = df.loc[linhas_vazias, :]


# Conversao de texto/categoria/string para numeros inteiros
# Convert the 'Delivery_person_Age' column to numeric values
df['Delivery_person_Age'] = pd.to_numeric(df['Delivery_person_Age'])

# Conversao de texto/categoria/strings para numero decimais
df['Delivery_person_Ratings'] = df['Delivery_person_Ratings'].astype( float )

# Conversão de texto para data
# Remoção da última linha do DataFrame ( formato errado, apenas %d-%m )
df.drop(df.tail(1).index, inplace = True)
df['Order_Date'] = pd.to_datetime( df['Order_Date'], format='%d-%m-%Y' )

# Remove as linhas da coluna multiple deliveries que tenham o conteudo igual a 'NaN '
linhas_vazias = df['multiple_deliveries'] != 'NaN '
df= df.loc[linhas_vazias, :]
df['multiple_deliveries'] = df['multiple_deliveries'].astype( int )

# Comando para remover o texto de números
df = df.reset_index( drop=True )
for i in range( len( df ) ):
    df.loc[i, 'Time_taken(min)'] = re.findall( r'\d+', df.loc[i, 'Time_taken(min)'] ) #Regex que resgata .findall() no texto que deseja remover

# Convert the 'Time_taken(min)' column to a string type
df['Time_taken(min)'] = df['Time_taken(min)'].apply(str)
df['Time_taken(min)'] = df['Time_taken(min)'].astype(int)

# Removendo os espaços dentro de strings/texto/object
df.loc[:, 'ID'] = df.loc[:, 'ID'].str.strip()
df.loc[:, 'Road_traffic_density'] = df.loc[:, 'Road_traffic_density'].str.strip()
df.loc[:, 'Type_of_order'] = df.loc[:, 'Type_of_order'].str.strip()
df.loc[:, 'Type_of_vehicle'] = df.loc[:, 'Type_of_vehicle'].str.strip()
df.loc[:, 'City'] = df.loc[:, 'City'].str.strip()
df.loc[:, 'Festival'] = df.loc[:, 'Festival'].str.strip()

In [ ]:
# Check group sizes
print(df.groupby('City')['Time_taken(min)'].size())

City
Metropolitian    31862
Semi-Urban         152
Urban             9404
Name: Time_taken(min), dtype: int64


In [ ]:
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,30


In [ ]:
df.dtypes

ID                                     object
Delivery_person_ID                     object
Delivery_person_Age                     int64
Delivery_person_Ratings               float64
Restaurant_latitude                   float64
Restaurant_longitude                  float64
Delivery_location_latitude            float64
Delivery_location_longitude           float64
Order_Date                     datetime64[ns]
Time_Orderd                            object
Time_Order_picked                      object
Weatherconditions                      object
Road_traffic_density                   object
Vehicle_condition                       int64
Type_of_order                          object
Type_of_vehicle                        object
multiple_deliveries                     int64
Festival                               object
City                                   object
Time_taken(min)                         int64
dtype: object

In [ ]:
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,30


In [ ]:
df.dtypes

ID                                     object
Delivery_person_ID                     object
Delivery_person_Age                     int64
Delivery_person_Ratings               float64
Restaurant_latitude                   float64
Restaurant_longitude                  float64
Delivery_location_latitude            float64
Delivery_location_longitude           float64
Order_Date                     datetime64[ns]
Time_Orderd                            object
Time_Order_picked                      object
Weatherconditions                      object
Road_traffic_density                   object
Vehicle_condition                       int64
Type_of_order                          object
Type_of_vehicle                        object
multiple_deliveries                     int64
Festival                               object
City                                   object
Time_taken(min)                         int64
dtype: object

# Resposta as perguntas de negócio

# 1.0 Visão Empresa

In [ ]:
df.head(30)

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,30
5,0x9bb4,HYDRES09DEL03,22,4.8,17.431668,78.408321,17.461668,78.438321,2022-03-11,21:20:00,21:30:00,conditions Cloudy,Jam,0,Buffet,motorcycle,1,No,Urban,26
6,0x95b4,RANCHIRES15DEL01,33,4.7,23.369746,85.339820,23.479746,85.449820,2022-03-04,19:15:00,19:30:00,conditions Fog,Jam,1,Meal,scooter,1,No,Metropolitian,40
7,0x9eb2,MYSRES15DEL02,35,4.6,12.352058,76.606650,12.482058,76.736650,2022-03-14,17:25:00,17:30:00,conditions Cloudy,Medium,2,Meal,motorcycle,1,No,Metropolitian,32
8,0x1102,HYDRES05DEL02,22,4.8,17.433809,78.386744,17.563809,78.516744,2022-03-20,20:55:00,21:05:00,conditions Stormy,Jam,0,Buffet,motorcycle,1,No,Metropolitian,34
9,0xcdcd,DEHRES17DEL01,36,4.2,30.327968,78.046106,30.397968,78.116106,2022-02-12,21:55:00,22:10:00,conditions Fog,Jam,2,Snack,motorcycle,3,No,Metropolitian,46


In [ ]:
cols = ['City', 'Time_taken(min)']
df_aux =  df.loc[0:10, cols].groupby( 'City' ).agg( {'Time_taken(min)': ['mean','std']} )

df_aux.columns = ['avg_time', 'std_time']

df_aux = df_aux.reset_index()

df_aux

,City,avg_time,std_time
0,Metropolitian,32.375000,8.192985
1,Urban,25.333333,1.154701


## 1. Quantidade de pedidos por dia.

In [ ]:
df1 = df.copy()
# Quantidade de pedidos por dia
df_aux = df1.loc[:, ['ID', 'Order_Date']].groupby( 'Order_Date' ).count().reset_index()
df_aux.columns = ['order_date', 'qtde_entregas']
# gráfico
px.bar( df_aux, x='order_date', y='qtde_entregas' )

## 2. Quantidade de pedidos por semana.

In [ ]:
df2 = df1.copy()

# Quantidade de pedidos por Semana
df2['week_of_year'] = df2['Order_Date'].dt.strftime( "%U" )
df_aux = df2.loc[:, ['ID', 'week_of_year']].groupby( 'week_of_year' ).count().reset_index()

# gráfico
px.line( df_aux, x='week_of_year', y='ID' )

## 3. Distribuição dos pedidos por tipo de tráfego.

In [ ]:
df_aux = df2.loc[:, ['ID', 'Road_traffic_density']].groupby( 'Road_traffic_density' ).count().reset_index()
df_aux = df_aux.loc[df_aux['Road_traffic_density'] != "NaN", :]
df_aux['perc_ID'] = 100 * ( df_aux['ID'] / df_aux['ID'].sum() )

# gráfico
px.pie( df_aux, values='perc_ID', names='Road_traffic_density' )

## 4. Comparação do volume de pedidos por cidade e tipo de tráfego.

In [ ]:
# agrupamento e contagem das colunas 
df_aux = df2.loc[:, ['ID', 'City', 'Road_traffic_density']].groupby( ['City','Road_traffic_density'] ).count().reset_index()
# removendo o NaN da coluna Road_traffic_density
df_aux = df_aux.loc[df_aux['Road_traffic_density'] != "NaN", :]
df_aux = df_aux.loc[df_aux['City'] != "NaN", :]
# comparação percentual
df_aux['perc_ID'] = 100 * ( df_aux['ID'] / df_aux['ID'].sum() )
# gráfico
px.pie( df_aux, values='perc_ID', names='Road_traffic_density' )

In [ ]:
# gráfico de bolha
px.scatter( df_aux, x='City', y='Road_traffic_density', size='ID',color='City')

## 5. A localização central de cada cidade por tipo de tráfego.

In [ ]:
# Quantidade de pedidos por entregador por Semana
# Quantas entregas na semana / Quantos entregadores únicos por semana
df_aux1 = df2.loc[:, ['ID', 'week_of_year']].groupby( 'week_of_year' ).count().reset_index()
df_aux2 = df2.loc[:, ['Delivery_person_ID', 'week_of_year']].groupby( 'week_of_year').nunique().reset_index()
df_aux = pd.merge( df_aux1, df_aux2, how='inner' )
df_aux['order_by_delivery'] = df_aux['ID'] / df_aux['Delivery_person_ID']
# gráfico
px.line( df_aux, x='week_of_year', y='order_by_delivery' )

## 6. A localização central de cada cidade por tipo de tráfego.

In [ ]:
data_plot = df1.loc[:, ['City', 'Road_traffic_density', 'Delivery_location_latitude', 'Delivery_location_longitude']].groupby( ['City', 'Road_traffic_density']).median().reset_index()
data_plot = data_plot[data_plot['City'] != 'NaN']
data_plot = data_plot[data_plot['Road_traffic_density'] != 'NaN']
# Desenhar o mapa
map_ = folium.Map( zoom_start=11 )
for index, location_info in data_plot.iterrows():
    folium.Marker( [location_info['Delivery_location_latitude'],
                    location_info['Delivery_location_longitude']],popup=location_info[['City', 'Road_traffic_density']] ).add_to( map_ )
map_

### Exercicios
1. Desenhar um gráfico de pizza com a média de idade dos entregadores por
cidade
2. Desenhe um gráfico de linha, mostrando o número total de entregas diárias
feitas por densidade de tráfego igual a “Low” e “Jam”
3. Desenhe um gráfico de barras, mostrando as avaliações médias das
entregas por semana.
4. Desenhe um gráfico de pizza, mostrando a média de avaliações das
entregas feitas por condições climáticas?

# 2.0 Visão do Entregador

## 1. A menor e maior idade dos entregadores.

In [ ]:
df2 = df.copy()

In [ ]:
# A maior idade dos entregadores
print( 'A maior idade eh {}'.format( df2.loc[: , 'Delivery_person_Age'].max() ) )

# A menor idade dos entregadores
print( 'A menor idade eh {}'.format( df2.loc[: , 'Delivery_person_Age'].min() ) )

A maior idade eh 39
A menor idade eh 20


## 2. A pior e a melhor condição de veículos.

In [ ]:
# A melhor condição de veículos
print( 'A melhor condição de veículos eh {}'.format( df2.loc[: , 'Vehicle_condition'].max() ) )

# A pior condição de veículos
print( 'A pior condição de veículos {}'.format( df2.loc[: , 'Vehicle_condition'].min() ) )

A melhor condição de veículos eh 2
A pior condição de veículos 0


## 3- A avaliação média por entregador.

In [ ]:
df_avg_per_delivery = df2.loc[:, ['Delivery_person_ID','Delivery_person_Ratings']].groupby( 'Delivery_person_ID' ).mean().reset_index()
df_avg_per_delivery

,Delivery_person_ID,Delivery_person_Ratings
0,AGRRES010DEL01,4.761538
1,AGRRES010DEL02,4.671429
2,AGRRES010DEL03,4.575000
3,AGRRES01DEL01,4.522222
4,AGRRES01DEL02,4.700000
...,...,...
1315,VADRES19DEL02,4.632727
1316,VADRES19DEL03,4.670270
1317,VADRES20DEL01,4.620370
1318,VADRES20DEL02,4.591111


## 4- A avaliação média e o desvio padrão por tipo de tráfego.

In [ ]:
df2_avg_std_rating_by_traffic = df2.loc[ :, ['Delivery_person_Ratings','Road_traffic_density']].groupby('Road_traffic_density').agg( {'Delivery_person_Ratings': ['mean', 'std']} )

#df_avg_rating_by_Weatherconditions = ( df1.loc[ :, ['Delivery_person_Ratings','Weatherconditions']].groupby('Weatherconditions')
                                #.agg({'Delivery_person_Ratings':['mean','std']}) )
# mudanças das colunas
df2_avg_std_rating_by_traffic.columns = ['Delivery_mean', 'Delivery_std']

# reset do index
df2_avg_std_rating_by_traffic = df2_avg_std_rating_by_traffic.reset_index()
df2_avg_std_rating_by_traffic

,Road_traffic_density,Delivery_mean,Delivery_std
0,High,4.652230,0.273044
1,Jam,4.594019,0.329778
2,Low,4.645011,0.338080
3,Medium,4.660115,0.274249


## 5- A avaliação média e o desvio padrão por condições climáticas.

In [ ]:
# Avaliação média por condições climáticas
df2_avg_std_rating_by_Weatherconditions = ( df2.loc[ :, ['Delivery_person_Ratings','Weatherconditions']].groupby('Weatherconditions')
                                .agg({'Delivery_person_Ratings':['mean','std']}) )
# mudanças das colunas
df2_avg_std_rating_by_Weatherconditions.columns = ['Delivery_mean', 'Delivery_std']

# reset do index
df2_avg_std_rating_by_Weatherconditions = df2_avg_std_rating_by_Weatherconditions.reset_index()
df2_avg_std_rating_by_Weatherconditions

,Weatherconditions,Delivery_mean,Delivery_std
0,conditions Cloudy,4.651871,0.281197
1,conditions Fog,4.652930,0.275064
2,conditions Sandstorms,4.611748,0.310852
3,conditions Stormy,4.611819,0.313096
4,conditions Sunny,4.654868,0.396674
5,conditions Windy,4.616128,0.304565


## 6- Os 10 entregadores mais rápidos por cidade.

In [ ]:
df3 = df2.loc[ :, ['Delivery_person_ID', 'City', 'Time_taken(min)'] ].groupby( ['City', 'Delivery_person_ID' ] ).min().sort_values(['City', 'Time_taken(min)'], ascending=True ).reset_index()

df_aux01 = df3.loc[df3['City'] == 'Metropolitian', :].head(10)
df_aux02 = df3.loc[df3['City'] == 'Urban', :].head(10)
df_aux03 = df3.loc[df3['City'] == 'Semi-Urban', :].head(10)

# Juntar do aux1 ao aux3, uma em cima do outro - pd.concat()
df4 = pd.concat( [df_aux01, df_aux02, df_aux03] ).reset_index( drop=True )
df4

,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,AGRRES010DEL03,10
1,Metropolitian,AGRRES07DEL03,10
2,Metropolitian,AGRRES12DEL01,10
3,Metropolitian,AGRRES14DEL01,10
4,Metropolitian,AGRRES17DEL03,10
5,Metropolitian,ALHRES08DEL03,10
6,Metropolitian,ALHRES12DEL01,10
7,Metropolitian,ALHRES14DEL02,10
8,Metropolitian,ALHRES15DEL02,10
9,Metropolitian,ALHRES19DEL03,10


## 7. Os 10 entregadores mais lentos por cidade.

In [ ]:
df2 = df1.loc[ :, ['Delivery_person_ID', 'City', 'Time_taken(min)'] ].groupby( ['City', 'Delivery_person_ID' ] ).max().sort_values(['City', 'Time_taken(min)'], ascending=False ).reset_index()

df_aux01 = df2.loc[df2['City'] == 'Metropolitian', :].head(10)
df_aux02 = df2.loc[df2['City'] == 'Urban', :].head(10)
df_aux03 = df2.loc[df2['City'] == 'Semi-Urban', :].head(10)

# Juntar do aux1 ao aux3, uma em cima do outro - pd.concat()
df3 = pd.concat( [df_aux01, df_aux02, df_aux03] ).reset_index( drop=True )
df3

,City,Delivery_person_ID,Time_taken(min)
0,Metropolitian,ALHRES010DEL01,54
1,Metropolitian,AURGRES13DEL01,54
2,Metropolitian,BANGRES02DEL01,54
3,Metropolitian,BHPRES16DEL02,54
4,Metropolitian,CHENRES02DEL02,54
5,Metropolitian,CHENRES04DEL01,54
6,Metropolitian,CHENRES07DEL03,54
7,Metropolitian,CHENRES08DEL01,54
8,Metropolitian,CHENRES13DEL02,54
9,Metropolitian,COIMBRES010DEL03,54


#3.0 Visão do Restaurante

## 1- Quantidade de entregadores únicos.

In [ ]:
df6 = df.copy()

In [ ]:
delivery_unique = len( df6.loc[:, 'Delivery_person_ID'].unique() )

print( 'O numero de entregadores unicos eh: {}'.format( delivery_unique ) )

O numero de entregadores unicos eh: 1320


## 2- A distância média dos restaurantes e dos locais de entrega.

### Método SAPE
#### SA (Saída) -> Um valor ( distância média ) -> [ 10km, 5km, 20km, 1km] -> média -> 15.5km
#### P (Processo) -> Calcular a distância entre os pedidos entregues e os restaurantes -> [10km, 5km, 20km, 1km]
#### E (Entrada) -> Colunas de latitude e longitude tantos dos pedidos (fim) quanto dos restaurante (inicio) -> comando que calcula distancia.

In [ ]:
df6.columns

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Weatherconditions', 'Road_traffic_density',
       'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle',
       'multiple_deliveries', 'Festival', 'City', 'Time_taken(min)'],
      dtype='object')

In [ ]:
cols = ['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude']

df6['Distance'] = df6.loc[:, cols].apply( lambda x: haversine( (x['Restaurant_latitude'], x['Restaurant_longitude'] ), (x['Delivery_location_latitude'], x['Delivery_location_longitude'])), axis=1)

avg_distance = df6['Distance'].mean()

print( 'A distancia media das entregas eh: {:.2f} KM'.format( avg_distance ) )


A distancia media das entregas eh: 27.44 KM


## 3- O tempo médio e o desvio padrão de entrega por cidade.


In [ ]:
import numpy as np

In [ ]:
# Calculate the mean of the 'A' column
mean_a = df6['Time_taken(min)'].mean()

# Replace missing values in the 'A' column with the mean of the column
df6['Time_taken(min)'] = df6['Time_taken(min)'].fillna(mean_a)

In [ ]:
# Replace infinite values with the mean of the column
df6['Time_taken(min)'] = df6['Time_taken(min)'].replace([np.inf, -np.inf], np.nan)
df6['Time_taken(min)'] = df6['Time_taken(min)'].fillna(df6['Time_taken(min)'].mean())

# Group the 'Time_taken(min)' column by the 'City' column, calculate the mean and standard deviation for each group, and rename the resulting columns to 'avg_time' and 'std_time'
cols = ['City', 'Time_taken(min)']
df_aux = df6.loc[:, cols].groupby('City').agg({'Time_taken(min)': ['mean','std']})
df_aux.columns = ['avg_time', 'std_time']

# Reset the index so that the 'City' column becomes a regular column rather than an index column
df_aux = df_aux.reset_index()
df_aux

,City,avg_time,std_time
0,Metropolitian,27.427814,9.133391
1,Semi-Urban,49.710526,2.724992
2,Urban,23.209379,8.858049


In [ ]:
df6['Time_taken(min)'].isnull().sum() and df6['Time_taken(min)'].isin([np.inf, -np.inf]).sum()

0

In [ ]:
cols = ['City', 'Time_taken(min)']
df_aux =  df6.loc[:, cols].groupby( 'City' ).agg( {'Time_taken(min)': ['mean','std']} )

df_aux.columns = ['avg_time', 'std_time']

df_aux = df_aux.reset_index()

df_aux

,City,avg_time,std_time
0,Metropolitian,27.427814,9.133391
1,Semi-Urban,49.710526,2.724992
2,Urban,23.209379,8.858049


## 4- O tempo médio e o desvio padrão de entrega por cidade e tipo de pedido.

In [ ]:
df6.columns

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Weatherconditions', 'Road_traffic_density',
       'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle',
       'multiple_deliveries', 'Festival', 'City', 'Time_taken(min)',
       'Distance'],
      dtype='object')

In [ ]:
# Select the 'City' and 'Type_of_order' columns
cols = ['City', 'Type_of_order','Time_taken(min)']

# Group the DataFrame by 'City' and calculate the mean and standard deviation of the 'Type_of_order' column
df_aux = df.loc[:, cols].groupby( ['City', 'Type_of_order'] ).agg({'Time_taken(min)': ['mean', 'std']})

# Rename the columns
df_aux.columns = ['avg_time', 'std_time']

# Reset the index
df_aux = df_aux.reset_index()

df_aux

,City,Type_of_order,avg_time,std_time
0,Metropolitian,Buffet,27.299008,9.153107
1,Metropolitian,Drinks,27.322691,9.041655
2,Metropolitian,Meal,27.616383,9.214536
3,Metropolitian,Snack,27.467349,9.119747
4,Semi-Urban,Buffet,49.707317,2.731702
5,Semi-Urban,Drinks,49.625000,2.459347
6,Semi-Urban,Meal,50.300000,3.041665
7,Semi-Urban,Snack,49.408163,2.707385
8,Urban,Buffet,23.560652,9.056348
9,Urban,Drinks,23.311977,8.927314


## 5- O tempo médio e o desvio padrão de entrega por cidade e tipo de tráfego.

In [ ]:
cols = ['City', 'Time_taken(min)', 'Road_traffic_density']
df_aux =  df6.loc[:, cols].groupby( ['City', 'Road_traffic_density'] ).agg( {'Time_taken(min)': ['mean','std']} )

df_aux.columns = ['avg_time', 'std_time']

df_aux = df_aux.reset_index()

df_aux

,City,Road_traffic_density,avg_time,std_time
0,Metropolitian,High,28.140898,7.904645
1,Metropolitian,Jam,31.976991,9.476203
2,Metropolitian,Low,22.257675,6.794772
3,Metropolitian,Medium,27.728919,8.308069
4,Semi-Urban,High,50.125000,2.629956
5,Semi-Urban,Jam,49.841270,2.717095
6,Semi-Urban,Medium,47.400000,2.011080
7,Urban,High,24.305335,8.494842
8,Urban,Jam,27.993164,10.078271
9,Urban,Low,19.446809,6.319963


## 6- O tempo médio de entrega durantes os Festivais.

In [ ]:
cols = ['Time_taken(min)', 'Festival']
df_aux =  df6.loc[:, cols].groupby( ['Festival'] ).agg( {'Time_taken(min)': ['mean','std']} )

df_aux.columns = ['avg_time', 'std_time']

df_aux = df_aux.reset_index()

linhas_selecionadas = df_aux['Festival'] == 'Yes'
df_aux = df_aux.loc[linhas_selecionadas, :]

df_aux

,Festival,avg_time,std_time
1,Yes,45.518607,4.005399
